# 文献综述

在探索新主题时，一个常见的任务是进行文献综述。在这个例子中，我们将探索如何配置多代理团队来进行一个_简单的_文献综述。

- **Arxiv 搜索代理**：使用 Arxiv API 搜索与给定主题相关的论文并返回结果。
- **Google 搜索代理**：使用 Google 搜索 API 查找与给定主题相关的论文并返回结果。
- **报告代理**：根据搜索和股票分析代理收集的信息生成报告。

首先，让我们导入必要的模块。

In [2]:
from autogen_agentchat.agents import CodingAssistantAgent, ToolUseAssistantAgent
from autogen_agentchat.task import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_core.components.tools import FunctionTool
from autogen_ext.models import OpenAIChatCompletionClient

## 定义工具

接下来，我们将定义代理将用来执行任务的工具。在这种情况下，我们将定义一个简单的函数 `search_arxiv`，它将使用 `arxiv` 库来搜索与给定主题相关的论文。

最后，我们将把这些函数封装到一个 `FunctionTool` 类中，这样我们就可以在代理中将其用作工具。

注意：您需要根据需要设置适当的环境变量。

同时安装所需的库：

```bash
!pip install arxiv
```

In [9]:
import json


def google_search(query: str, num_results: int = 5, max_chars: int = 1500) -> list:  # type: ignore[type-arg]
    import os
    import time

    import requests
    from bs4 import BeautifulSoup
    from dotenv import load_dotenv

    load_dotenv()

    #api_key = os.getenv("GOOGLE_API_KEY")
    api_key = ""
    #search_engine_id = os.getenv("GOOGLE_SEARCH_ENGINE_ID")
    search_engine_id = ""
    
    if not api_key or not search_engine_id:
        raise ValueError("API key or Search Engine ID not found in environment variables")

    url = "https://www.googleapis.com/customsearch/v1"
    params = {"key": api_key, "cx": search_engine_id, "q": query, "num": num_results}

    response = requests.get(url, params=params)  # type: ignore[arg-type]

    if response.status_code != 200:
        print(response.json())
        raise Exception(f"Error in API request: {response.status_code}")

    results = response.json().get("items", [])

    def get_page_content(url: str) -> str:
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.content, "html.parser")
            text = soup.get_text(separator=" ", strip=True)
            words = text.split()
            content = ""
            for word in words:
                if len(content) + len(word) + 1 > max_chars:
                    break
                content += " " + word
            return content.strip()
        except Exception as e:
            print(f"Error fetching {url}: {str(e)}")
            return ""

    enriched_results = []
    for item in results:
        body = get_page_content(item["link"])
        enriched_results.append(
            {"title": item["title"], "link": item["link"], "snippet": item["snippet"], "body": body}
        )
        time.sleep(1)  # Be respectful to the servers

    return enriched_results


def arxiv_search(query: str, max_results: int = 5) -> list:  # type: ignore[type-arg]
    """
    Search Arxiv for papers and return the results including abstracts.
    """
    import arxiv

    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance)

    results = []
    for paper in client.results(search):
        results.append(
            {
                "title": paper.title,
                "authors": [author.name for author in paper.authors],
                "published": paper.published.strftime("%Y-%m-%d"),
                "abstract": paper.summary,
                "pdf_url": paper.pdf_url,
            }
        )

    # # Write results to a file
    with open('arxiv_search_results.json', 'w') as f:
        json.dump(results, f, indent=2)

    return results

In [10]:
google_search_tool = FunctionTool(
    google_search, description="Search Google for information, returns results with a snippet and body content"
)
arxiv_search_tool = FunctionTool(
    arxiv_search, description="Search Arxiv for papers related to a given topic, including abstracts"
)

## 定义代理

接下来，我们将定义执行任务的代理。

In [11]:
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    temperature=1,
    api_key="sk-", # Optional if you have an OPENAI_API_KEY env variable set.
)

google_search_agent = ToolUseAssistantAgent(
    name="Google_Search_Agent",
    registered_tools=[google_search_tool],
    model_client=model_client,
    description="An agent that can search Google for information, returns results with a snippet and body content",
    system_message="You are a helpful AI assistant. Solve tasks using your tools.",
)

arxiv_search_agent = ToolUseAssistantAgent(
    name="Arxiv_Search_Agent",
    registered_tools=[arxiv_search_tool],
    model_client=model_client,
    description="An agent that can search Arxiv for papers related to a given topic, including abstracts",
    system_message="You are a helpful AI assistant. Solve tasks using your tools. Specifically, you can take into consideration the user's request and craft a search query that is most likely to return relevant academi papers.",
)


report_agent = CodingAssistantAgent(
    name="Report_Agent",
    model_client=model_client,
    description="Generate a report based on a given topic",
    system_message="You are a helpful assistant. Your task is to synthesize data extracted into a high quality literature review including CORRECT references. You MUST write a final report that is formatted as a literature review with CORRECT references.  Your response should end with the word 'TERMINATE'",
)

termination = TextMentionTermination("TERMINATE")
team = RoundRobinGroupChat(
    participants=[google_search_agent, arxiv_search_agent, report_agent], termination_condition=termination
)

result = await team.run(
    task="Write a literature review on building AI Web Agents",
)

print(result)

/tmp/ipykernel_5636/2366287184.py:7: DeprecationWarning: ToolUseAssistantAgent is deprecated. Use AssistantAgent instead.
  google_search_agent = ToolUseAssistantAgent(
/tmp/ipykernel_5636/2366287184.py:15: DeprecationWarning: ToolUseAssistantAgent is deprecated. Use AssistantAgent instead.
  arxiv_search_agent = ToolUseAssistantAgent(
/tmp/ipykernel_5636/2366287184.py:24: DeprecationWarning: CodingAssistantAgent is deprecated. Use AssistantAgent instead.
  report_agent = CodingAssistantAgent(


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a literature review on building AI Web Agents'), ToolCallMessage(source='Google_Search_Agent', models_usage=RequestUsage(prompt_tokens=119, completion_tokens=23), content=[FunctionCall(id='call_oPyBpmMyN7fDyImzHeqjwEFA', arguments='{"query":"AI Web Agents literature review","num_results":5}', name='google_search')]), ToolCallResultMessage(source='Google_Search_Agent', models_usage=None, content=[FunctionExecutionResult(content='[{\'title\': "What\'s the best AI research agent you\'ve used? : r/artificial", \'link\': \'https://www.reddit.com/r/artificial/comments/1aijicr/whats_the_best_ai_research_agent_youve_used/\', \'snippet\': \'Feb 4, 2024 ... 12 votes, 15 comments. Looking for a Research tool that is able to complete in depth research tasks through browsing online.\', \'body\': "You\'ve been blocked by network security. To continue, log in to your Reddit account or use your developer token If you thi